<a href="https://colab.research.google.com/github/boneeyah/SMU/blob/main/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [18]:
from pandas.io.parsers.readers import read_csv
df = read_csv("https://raw.githubusercontent.com/boneeyah/SMU/main/netflix_titles.csv")

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [20]:
df = df.drop(index=df[df['country'].isnull()].index)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7976 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7976 non-null   object
 1   type          7976 non-null   object
 2   title         7976 non-null   object
 3   director      5751 non-null   object
 4   cast          7305 non-null   object
 5   country       7976 non-null   object
 6   date_added    7967 non-null   object
 7   release_year  7976 non-null   int64 
 8   rating        7973 non-null   object
 9   duration      7973 non-null   object
 10  listed_in     7976 non-null   object
 11  description   7976 non-null   object
dtypes: int64(1), object(11)
memory usage: 810.1+ KB


In [21]:
colors = {
    'background':'#111111',
    'text':'#7fdbff'
}

In [8]:
df1 = df[['country','type','show_id']]

df1 = df1[['country','show_id']].groupby(by='country',as_index=False).count()
fig1 = px.scatter_geo(data_frame=df1,locations='country',size='show_id',locationmode='country names',width = 900)
fig1.update_layout(plot_bgcolor=colors['background'],paper_bgcolor=colors['background'],font_color = 'white')
fig1.update_geos(bgcolor=colors['background'],landcolor='#747A76',lakecolor='black',showcountries=True)

In [9]:
df2 = df[['type','country','show_id','release_year']].groupby(by=['release_year','type'],as_index=False).count()
fig2 = px.line(data_frame=df2,x='release_year',y='show_id',color='type',width = 900)
fig2.update_layout(plot_bgcolor=colors['background'],paper_bgcolor=colors['background'],font_color = 'white')

In [10]:
df3 = df[['rating','type','show_id']]
df3 = df3.groupby(['rating','type'],as_index=False).count()
fig3 = px.treemap(data_frame=df3,path=[px.Constant('Rating'),'type','rating'],values='show_id',color_discrete_map={'(?)':'#747A76','Movie':'blue','TV Show':'red'},width = 1800)
fig3.update_layout(paper_bgcolor=colors['background'],font_color='white')

In [11]:
app = JupyterDash(__name__)
app.layout = html.Div(style = {'backgroundColor':colors['background']},children = [
    html.H1('Netflix Catalog - Movies and Shows',style = {'color':colors['text']}),
    html.Div([
      html.Div([dcc.Graph(id='graph2',figure=fig1)],style={'display':'inline-block'}),
      html.Div([dcc.Graph(id='graph3',figure=fig2)],style={'display':'inline-block'})
    ]),
    html.Div([dcc.Graph(id='graph4',figure=fig3)],style={'display':'inline-block'})
    ])


if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

Testing dropdown

In [28]:
external_stylesheets = ['https://raw.githubusercontent.com/boneeyah/SMU/main/dash.css']

In [32]:
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
app.layout = html.Div(style = {'backgroundColor':colors['background']},children = [
    html.H1('Netflix Catalog - Movies and Shows',style = {'color':colors['text']}),
    html.Div([
      html.Div([dcc.Graph(id='graph1')],style={'display':'inline-block'}),
      html.Div([dcc.Graph(id='graph2')],style={'display':'inline-block'})
    ]),
    html.Div([dcc.Graph(id='graph3')],style={'display':'inline-block'}),
    html.Div([
        dcc.Dropdown(
            df['type'].unique(),
            'Content Type',
            id='content-filter'
        )
        
    ])
    ])


if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [14]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [16]:

app = JupyterDash()
### defining the HTML component
app.layout = html.Div(children=[html.Div("Welcome to the dashboard",style= {   "color": "white",
                                                      "text-align": "center","background-color": "blue",
                                                      "border-style": "dotted","display":"inline-block","width":"80%"
                                                      
                                                    }),
                       html.Div(dcc.Dropdown(id = "drop_down_1" ,options= options_list , value= 'good'
                                                       ),style= {
                                                      "color": "green",
                                                      "text-align": "center","background-color": "darkorange",
                                                      "border-style": "dotted","display":"inline-block","width":"20%"
                                                      
                                                    }),
                       html.Div(children=[html.P(
                            id="map-title",
                            children = "Forecast and validation for Facility ",
                        ), html.Div(dcc.Graph(id ="plot_area"))
                                                       ],style= {
                                                      "color": "black",
                                                      "text-align": "center","background-color": "yellow",
                                                      "border-style": "dotted","display":"inline-block","width":"75%",
                                                                                                            
                                                    })],style={"width":"100%",'paffing':10})

NameError: ignored